In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv1D, Embedding, Activation, Flatten
from keras.optimizers import Adam, SGD
import numpy as np
import re
import preprocessing as preproc
import time
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import helpers
import models
from tqdm import tqdm
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


In [2]:
start_time = time.time()

In [ ]:
# # baseline
# def create_baseline():
# # create model
#     model = Sequential()
#     model.add(Dense(60, input_dim=60, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(30, kernel_initializer='normal', activation='relu'))
#     model.add(Dense(1, kernel_initializer='normal', activation='sigmoid'))
#     sgd = SGD(lr=0.01, momentum=0.8, decay=0.0, nesterov=False)
#     model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
#     return model

In [ ]:
preproc.do_preprocessing('../data/twitter-datasets/train_pos.txt')
preproc.do_preprocessing('../data/twitter-datasets/train_neg.txt')

In [2]:
lines, y = preproc.return_processed_trainset_and_y(True)
len(lines)

2500000

In [3]:
vectorizer = TfidfVectorizer(stop_words=None, ngram_range=(1,2), sublinear_tf=True, max_features=100)
X = vectorizer.fit_transform(lines).toarray()

In [4]:
type(X)

numpy.ndarray

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
optimizer = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

In [ ]:
optimizer = SGD(lr=0.04, momentum=0.08, decay=0.0001, nesterov=True)

In [ ]:
X.shape

In [9]:
def create_model():
    model = Sequential()
    model.add(Dense(units=128, input_dim=X.shape[1], activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(units=64, activation='relu'))
#     model.add(Dropout(0.1))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['acc'])
    return model
model = create_model()

In [ ]:
result = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=32, verbose=2)

In [ ]:
np.mean(result.history['val_acc'])

In [ ]:
result.history()

In [ ]:
#best
np.mean(result.history['val_acc'])

In [ ]:
scores = model.evaluate(X, y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

In [12]:
model.fit_generator(generator=batch_generator(X_train, y_train, 32),
                   epochs=5, validation_data=(X_test,y_test),
                    steps_per_epoch=int(X_train.shape[0]/32))

Epoch 1/5
62500/62500 [==============================] - 153s 2ms/step - loss: 0.5413 - acc: 0.7129 - val_loss: 0.5411 - val_acc: 0.7123
Epoch 2/5
62500/62500 [==============================] - 154s 2ms/step - loss: 0.5411 - acc: 0.7132 - val_loss: 0.5409 - val_acc: 0.7124
Epoch 3/5
62500/62500 [==============================] - 155s 2ms/step - loss: 0.5409 - acc: 0.7132 - val_loss: 0.5407 - val_acc: 0.7123
Epoch 4/5
62500/62500 [==============================] - 153s 2ms/step - loss: 0.5407 - acc: 0.7134 - val_loss: 0.5406 - val_acc: 0.7125
Epoch 5/5
62500/62500 [==============================] - 151s 2ms/step - loss: 0.5406 - acc: 0.7139 - val_loss: 0.5405 - val_acc: 0.7125


In [ ]:
a = batch_generator(X,y,32)

In [ ]:
next(a)

In [10]:
def batch_generator(X, y, batch_size):
    batch_per_epoch = int(X.shape[0]/batch_size)
    index = np.arange(np.shape(y)[0])
    batches_x = [X[batch_size*i:batch_size*(i+1)] for i in range(batch_per_epoch)]
    batches_y = [y[batch_size*i:batch_size*(i+1)] for i in range(batch_per_epoch)]
    while(True):
        for counter in range(len(batches_x)):
            yield batches_x[counter], batches_y[counter]

400 features
Me 10000 data:
# 78.35 Adam:decay 0.002 lr 0.05 epochs 100
# 73 Adam:decay 0.02

Me 20000 data:
# 77.23 Adam:decay 0.002 lr 0.05 epochs 100

Me 200000 data:
# 78.22 Adam:decay 0.002 lr 0.05 epochs 100

Ola

 78.94% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)
 
 79.40% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
 
Features 1000

 81% adam = Adam(lr=0.03, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)
  
 83 % adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.0001, amsgrad=False)

In [ ]:
lines1 = lines[:10000] + lines[100000:110000]
y1 = np.concatenate((y[:10000], y[100000:110000]), axis=0)
y1 = y1.tolist()
print(len(lines1))
print(len(y1))

In [ ]:
np.unique(y)

84.5% adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)

82.5 adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.005, amsgrad=False)

83.49 adam = Adam(lr=0.08, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.005, amsgrad=False)

# best model so far
adam = Adam(lr=0.02, beta_1=0.9, beta_2=0.99, epsilon=None, decay=0.001, amsgrad=False)
model = Sequential()
model.add(Dense(units=64, input_dim=X.shape[1], activation='relu'))
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
model.summary()